In [ ]:
#Dectectron library is needed to be installed for layout parser. Windows installation is very complex so if you want to execute this notebook
#Easiest way is to install WSL on windows and install detectron on virtual ubuntu and run this notebook.
#Executing on linux machine is the simplest way.

In [4]:
import torchvision.ops.boxes as bops 

In [5]:
import pdf2image
import numpy as np
import layoutparser as lp
import torch
import numpy as np
from PIL import Image

In [6]:
model = lp.Detectron2LayoutModel(config_path = "lp://PubLayNet/mask_rcnn_X_101_32x8d_FPN_3x/config",
                                 extra_config=["MODEL.ROI_HEADS.SCORE_THRESH_TEST", 0.5],
                                 label_map={0: "Text", 1: "Title", 2: "List", 3:"Table", 4:"Figure"})

The checkpoint state_dict contains keys that are not used by the model:
  proposal_generator.anchor_generator.cell_anchors.{0, 1, 2, 3, 4}


In [7]:
#Input pdf file path and convert it to image
pdf_file= '/mnt/c/Users/s.shrikant.bhurke/OneDrive - Accenture/Sample 100 files - Lease & amendment/Lease Agreement.pdf'
#pdf_file= r'C:\Users\s.shrikant.bhurke\OneDrive - Accenture\Sample 100 files - Lease & amendment\1215 Lease.pdf' # Adjust the filepath of your input image accordingly
img = pdf2image.convert_from_path(pdf_file)

In [ ]:
#Using azure ocr xml to get the text and the azure xml output has co-ordinates in twips so converting to pixel as layout parser considers pixel co-ordinates for bounding boxes.
import xml.etree.ElementTree as ET
from lxml import etree
from lxml.etree import XMLSyntaxError

def convert_twip_to_pixel(tz):
    twip_factor = 0.0666666667
    l = (float(tz.attrib['l'])*200)/1440
    t = (float(tz.attrib['t'])*200)/1440
    r = (float(tz.attrib['r'])*200)/1440
    b = (float(tz.attrib['b'])*200)/1440
    return l,t,r,b


def get_word_list(xml_string, PARA_SPACE=300):
    parser = etree.XMLParser(ns_clean=True, recover=True, encoding='utf-8')
    tree = etree.ElementTree(etree.fromstring(xml_string, parser=parser))

    root = tree.getroot()
    text_content = ""
    page_num = 0
    all_word_list = {}
    for page in root.iter('{http://www.scansoft.com/omnipage/xml/ssdoc-schema3.xsd}page'):
        word_list = []
        for tz in page.iter('{http://www.scansoft.com/omnipage/xml/ssdoc-schema3.xsd}textZone'):
            tz_content = ' '.join(filter(None, [tx.strip() for tx in tz.itertext()]))
            #print(tz_content)
            #l,t,r,b = convert_twip_to_pixel(tz)
            #word_list.append([tz_content, l, t, r, b])
            for w in tz.iter():
                if len(w.text.strip()) > 0:
                    l,t,r,b = convert_twip_to_pixel(w)
                    word_list.append([w.text, l, t, r, b])
                    ##word_list.append([w.text, float(w.attrib['l']), float(w.attrib['t']), float(w.attrib['r']), float(w.attrib['b'])])
        all_word_list.update({page_num: word_list})
        # print(page_num,word_list)
        # print("...................",page_num)
        page_num = page_num + 1
    # print(all_word_list)
    return all_word_list


def return_word(xml_path):
    all_word_list = {}
    with open(xml_path, encoding='utf8') as f:
        all_word_list = get_word_list(f.read())
        #print("all_word_list",all_word_list)
    return all_word_list


In [9]:
# Mapping the bounding box co-ordinates from layout parser and extracting the text from azure ocr xml.
xml_path = r"/mnt/c/Users/s.shrikant.bhurke/OneDrive - Accenture/CNW/Sample 100 files - Lease & amendment/Lease Agreement_layout.xml"
all_word_list = return_word(xml_path)
paragraph_list = []
text_type = []
#count = 0
for i in range(len(img)):
    page_image = img[i]
    img1 = np.array(page_image)
    layout_result = model.detect(page_image)

    # lp.draw_box(img, layout_result,  box_width=5, box_alpha=0.2, show_element_type=True)
    text_blocks = lp.Layout([b for b in layout_result])

    #lp.draw_box(page_image, text_blocks,  box_width=5, box_alpha=0.2, show_element_type=True, show_element_id=True)
    text_blocks = lp.Layout([b for b in layout_result if b.type in ['Text', 'Title', "List", 'Table']])
    figure_blocks = lp.Layout([b for b in layout_result if b.type=='Figure'])

    text_blocks = lp.Layout([b for b in text_blocks if not any(b.is_in(b_fig) for b_fig in figure_blocks)])
    #to sort the bounding box from page start to end

    w, h  = page_image.size

    left_interval = lp.Interval(0, w/2*1.05, axis='x').put_on_canvas(page_image)

    left_blocks = text_blocks.filter_by(left_interval, center=True)
    left_blocks.sort(key = lambda b:b.coordinates[1], inplace=True)

    right_blocks = [b for b in text_blocks if b not in left_blocks]
    right_blocks.sort(key = lambda b:b.coordinates[1])

    # And finally combine the two list and add the index
    # according to the order
    text_blocks = lp.Layout([b.set(id = idx) for idx, b in enumerate(left_blocks + right_blocks)])

    text_blocks = sorted(text_blocks._blocks, key=lambda block: block.block.y_1)

    
    print("text_blocks",text_blocks)
    #print("-------text_blocks--------",text_blocks)
    for block in text_blocks:
        #segment_image = (block.pad(left=5, right=5, top=5, bottom=5).crop_image(img1))
            # add padding in each image segment can help
            # improve robustness
        x_1 = block.block.x_1  
        y_1 = block.block.y_1
        x_2 = block.block.x_2
        y_2 = block.block.y_2
        paragraph = ""
        print("---Block----",x_1, y_1, x_2, y_2)
        values = all_word_list[i]
        for textzone in values:
            if textzone[1] >= x_1 and textzone[3] <= x_2 and textzone[2] >= y_1 and textzone[4] <= y_2:
                paragraph += textzone[0] + " "
                print("------------paragraph----------",paragraph)
        paragraph_list.append(paragraph)
        text_type.append(block.type)
        #print("Block",block.type)
    
        #print("------paragraph------",paragraph)
        #
        # block.set(text=paragraph, inplace=True)
    #print("paragraph_list",paragraph_list)

    #for txt in text_blocks.get_texts():
        #print(txt, end='\n---\n')
#print("paragraph_list",paragraph_list)

    

/home/shraddha/miniconda3/envs/section_seg/lib/python3.10/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3483.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


------------------------------- Layout(_blocks=[TextBlock(block=Rectangle(x_1=172.1931610107422, y_1=230.52845764160156, x_2=1588.146484375, y_2=360.8491516113281), text=None, id=None, type=Title, parent=None, next=None, score=0.9085199236869812), TextBlock(block=Rectangle(x_1=153.043701171875, y_1=756.086669921875, x_2=1583.022705078125, y_2=920.4722290039062), text=None, id=None, type=Text, parent=None, next=None, score=0.8203796744346619), TextBlock(block=Rectangle(x_1=165.13165283203125, y_1=1469.2979736328125, x_2=1574.015869140625, y_2=1595.5947265625), text=None, id=None, type=Text, parent=None, next=None, score=0.7653172016143799), TextBlock(block=Rectangle(x_1=140.62022399902344, y_1=1866.892578125, x_2=1591.7056884765625, y_2=2039.139404296875), text=None, id=None, type=Text, parent=None, next=None, score=0.7589321136474609), TextBlock(block=Rectangle(x_1=316.3785095214844, y_1=513.677490234375, x_2=1441.8779296875, y_2=623.0610961914062), text=None, id=None, type=Title, pare

In [28]:
img = img[3]
img1 = np.array(img)

In [ ]:
layout_result = model.detect(img)

# lp.draw_box(img, layout_result,  box_width=5, box_alpha=0.2, show_element_type=True)
text_blocks = lp.Layout([b for b in layout_result])

lp.draw_box(img
            , text_blocks,  box_width=5, box_alpha=0.2, show_element_type=True, show_element_id=True)

In [37]:
layout_result

Layout(_blocks=[TextBlock(block=Rectangle(x_1=106.40232849121094, y_1=1196.302734375, x_2=1567.594970703125, y_2=2133.689453125), text=None, id=None, type=Figure, parent=None, next=None, score=0.9021470546722412), TextBlock(block=Rectangle(x_1=148.9532470703125, y_1=188.73300170898438, x_2=1582.36962890625, y_2=405.4966125488281), text=None, id=None, type=Text, parent=None, next=None, score=0.9013590812683105), TextBlock(block=Rectangle(x_1=154.1140594482422, y_1=626.370849609375, x_2=1581.835205078125, y_2=794.349365234375), text=None, id=None, type=Text, parent=None, next=None, score=0.8770572543144226), TextBlock(block=Rectangle(x_1=145.6768341064453, y_1=426.60101318359375, x_2=1577.3323974609375, y_2=590.841796875), text=None, id=None, type=Text, parent=None, next=None, score=0.8742883801460266), TextBlock(block=Rectangle(x_1=162.90737915039062, y_1=824.5476684570312, x_2=1592.5294189453125, y_2=950.5324096679688), text=None, id=None, type=Text, parent=None, next=None, score=0.847

In [30]:
text_blocks = lp.Layout([b for b in layout_result if b.type in ['Text', "List"]])
figure_blocks = lp.Layout([b for b in layout_result if b.type=='Figure'])

In [31]:
text_blocks = lp.Layout([b for b in text_blocks \
                   if not any(b.is_in(b_fig) for b_fig in figure_blocks)])

In [32]:
w, h  = img.size

left_interval = lp.Interval(0, w/2*1.05, axis='x').put_on_canvas(img)

left_blocks = text_blocks.filter_by(left_interval, center=True)
left_blocks.sort(key = lambda b:b.coordinates[1], inplace=True)

right_blocks = [b for b in text_blocks if b not in left_blocks]
right_blocks.sort(key = lambda b:b.coordinates[1])

# And finally combine the two list and add the index
# according to the order
text_blocks = lp.Layout([b.set(id = idx) for idx, b in enumerate(left_blocks + right_blocks)])

In [34]:
text_blocks

Layout(_blocks=[TextBlock(block=Rectangle(x_1=148.9532470703125, y_1=188.73300170898438, x_2=1582.36962890625, y_2=405.4966125488281), text=None, id=0, type=Text, parent=None, next=None, score=0.9013590812683105), TextBlock(block=Rectangle(x_1=145.6768341064453, y_1=426.60101318359375, x_2=1577.3323974609375, y_2=590.841796875), text=None, id=1, type=Text, parent=None, next=None, score=0.8742883801460266), TextBlock(block=Rectangle(x_1=154.1140594482422, y_1=626.370849609375, x_2=1581.835205078125, y_2=794.349365234375), text=None, id=2, type=Text, parent=None, next=None, score=0.8770572543144226), TextBlock(block=Rectangle(x_1=162.90737915039062, y_1=824.5476684570312, x_2=1592.5294189453125, y_2=950.5324096679688), text=None, id=3, type=Text, parent=None, next=None, score=0.8477199673652649), TextBlock(block=Rectangle(x_1=174.02711486816406, y_1=985.4822998046875, x_2=1568.0482177734375, y_2=1073.53515625), text=None, id=4, type=Text, parent=None, next=None, score=0.6070707440376282)

In [ ]:
xml_path = r"/mnt/c/Users/s.shrikant.bhurke/OneDrive - Accenture/CNW/Sample 100 files - Lease & amendment/Lease Agreement_layout.xml"
all_word_list = return_word(xml_path)
paragraph_list = []
print("-------text_blocks--------",text_blocks)
for block in text_blocks:
    segment_image = (block.pad(left=5, right=5, top=5, bottom=5).crop_image(img1))
        # add padding in each image segment can help
        # improve robustness
    x_1 = block.block.x_1  
    y_1 = block.block.y_1
    x_2 = block.block.x_2
    y_2 = block.block.y_2
    paragraph = ""
    for page in all_word_list:
        
        if page == 3:
            values = all_word_list[3]
            for textzone in values:
                #if textzone[1] <= x_2 and textzone[3] >= x_1 and textzone[2] <= y_2 and textzone[4] >= y_1:
                #x_midpoint = ((textzone[3]-textzone[1])/2)+textzone[1]
                #y_midpoint = ((textzone[4]-textzone[2])/2)+textzone[2]
                #if (x_midpoint >= x_1 and y_midpoint >= y_1 and x_midpoint <= x_2 and y_midpoint <= y_2):
                #if (x_midpoint >= x_1) and (x_midpoint <= x_1 + box_width) and (y_midpoint >= box_y) and (y_midpoint <= box_y + box_height):
                if textzone[1] >= x_1 and textzone[3] <= x_2 and textzone[2] >= y_1 and textzone[4] <= y_2:
                    paragraph += textzone[0] + " "
            break
    paragraph_list.append(paragraph)
    print("------paragraph------",paragraph)
    #
    # block.set(text=paragraph, inplace=True)
#print("paragraph_list",paragraph_list)

#for txt in text_blocks.get_texts():
    #print(txt, end='\n---\n')